In [76]:
# import relevant packages

import pandas as pd
import os
import re

from html import unescape
from sklearn.model_selection import train_test_split

pd.set_option('display.max_colwidth', None)

## Load raw datasets

In [77]:
df_dict = dict()

PATH = "/mounts/data/proj/faeze/data_efficient_hate/datasets/main/"

for file in sorted(os.listdir(PATH+'0_raw')):
    if 'measure' in file:
        continue
    if 'gahd' not in file:# and 'xdomain' not in file and 'implicit' not in file and 'xplain' not in file:
        continue
    if "ipynb" not in file:
        if file.endswith(".csv"):
            print(re.sub('\.csv$', '', file))
            df_dict[re.sub('\.csv$', '', file)] = pd.read_csv(f"{PATH}/0_raw/{file}", on_bad_lines='skip')
        else:
            print(re.sub('\.tsv$', '', file))
            df_dict[re.sub('\.tsv$', '', file)] = pd.read_csv(f"{PATH}/0_raw/{file}", on_bad_lines='skip', delimiter='\t')

gahd24_de


<>:12: SyntaxWarning: invalid escape sequence '\.'
<>:13: SyntaxWarning: invalid escape sequence '\.'
<>:15: SyntaxWarning: invalid escape sequence '\.'
<>:16: SyntaxWarning: invalid escape sequence '\.'
<>:12: SyntaxWarning: invalid escape sequence '\.'
<>:13: SyntaxWarning: invalid escape sequence '\.'
<>:15: SyntaxWarning: invalid escape sequence '\.'
<>:16: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_170674/2774132277.py:12: SyntaxWarning: invalid escape sequence '\.'
  print(re.sub('\.csv$', '', file))
/tmp/ipykernel_170674/2774132277.py:13: SyntaxWarning: invalid escape sequence '\.'
  df_dict[re.sub('\.csv$', '', file)] = pd.read_csv(f"{PATH}/0_raw/{file}", on_bad_lines='skip')
/tmp/ipykernel_170674/2774132277.py:15: SyntaxWarning: invalid escape sequence '\.'
  print(re.sub('\.tsv$', '', file))
/tmp/ipykernel_170674/2774132277.py:16: SyntaxWarning: invalid escape sequence '\.'
  df_dict[re.sub('\.tsv$', '', file)] = pd.read_csv(f"{PATH}/0_raw/{file}", on_bad_line

In [78]:
df_dict["implicit_en"]['label'].value_counts()

KeyError: 'implicit_en'

## Reformat columns
Need separate logic for different datasets. 1 is for hateful, 0 for non-hateful.

In [79]:
df_dict["implicit_en"]['label'].replace({0:1, 1:1, 2:0}, inplace=True)


KeyError: 'implicit_en'

In [80]:
df_dict["implicit_en"].label.value_counts()

KeyError: 'implicit_en'

In [81]:
# Dynabench 2021 / English
df_dict["dyn21_en"].label.replace({"hate":1, "nothate":0}, inplace=True)

# Founta 2018 / English
df_dict["fou18_en"].label.replace({'hateful': 1, "abusive": 0, "normal": 0, "spam": 0}, inplace = True)

# Kennedy 2020 / English
df_dict["ken20_en"].rename(columns={"label_hate_maj": "label"}, inplace=True)

# Fortuna 2019 / Portuguese
df_dict["for19_pt"].rename(columns={"hatespeech_comb": "label"}, inplace=True)

# Basile 2019 / Spanish
df_dict["bas19_es"].rename(columns={"HS": "label"}, inplace=True)

# Sanguinetti 2020 / Italian
df_dict["san20_it"].rename(columns={"hs": "label"}, inplace=True)

# Ousidhoum 2019 / Arabic & French
for d in ["ous19_ar", "ous19_fr"]:
    df_dict[d]["label"] = df_dict[d].sentiment.apply(lambda x: 1 if "hateful" in x else 0)
    # text was already cleaned in a way that conflicts with our later cleaning, so we align it here
    df_dict[d]["text"] = df_dict[d].tweet.apply(lambda x: x.replace("@url", "http"))

# HASOC 19, 20 and 21 / Hindi
for d in ["has19_hi", "has20_hi", "has21_hi"]:
    df_dict[d]["label"] = df_dict[d].task_2.apply(lambda x: 1 if x=="HATE" else 0)

KeyError: 'dyn21_en'

In [82]:

# drop redundant columns
for dataset in df_dict:
    if "split" in df_dict[dataset].columns:
        df_dict[dataset] = df_dict[dataset][["text", "label", "split"]]
    else:
        df_dict[dataset] = df_dict[dataset][["text", "label"]]

## Clean text

In [83]:
def clean_text(text):
    text = unescape(text)
    text = re.sub(r"@[A-Za-z0-9_-]+",'@user',text) # format expected by XLM-T
    text = re.sub(r"http\S+",'http',text) # format expected by XLM-T
    text = re.sub(r"\n",' ',text)
    text = re.sub(r"\t",' ',text)
    text = text.replace("[URL]", "http") # format expected by XLM-T
    text = text.strip()
    return text

for dataset in df_dict:
    df_dict[dataset]["text"] = df_dict[dataset].text.apply(lambda x: clean_text(x))

In [84]:
# boost Kennedy 2020 / English to have 50% hate (up from ca. 30%)
df_dict["ken20_en"] = pd.concat([df_dict["ken20_en"][df_dict["ken20_en"].label==1], df_dict["ken20_en"][df_dict["ken20_en"].label==0].sample(11596, random_state=123)]).sample(frac=1, random_state=123)

# boost Founta 2018 / English to have 22% hate, which is max possible (up from ca. 5%)
df_dict["fou18_en"] = pd.concat([df_dict["fou18_en"][df_dict["fou18_en"].label==1], df_dict["fou18_en"][df_dict["fou18_en"].label==0].sample(17600, random_state=123)]).sample(frac=1, random_state=123)


KeyError: 'ken20_en'

In [85]:
df_dict["fou18_en"]["text"]

KeyError: 'fou18_en'

In [86]:
df_dict["gahd24_de"]


,text,label,split
0,"Ich denke, alles was man tun kann, ist, es zu ignorieren, so Brown.",0,train
1,"Es sind beides unsere Hunde, aber den schwarzen mag ich nicht so.",0,dev
2,Ich liebe Inder,0,train
3,Und dazu kommt auch noch die »Mode« LGBTQ.,1,dev
4,"Jeder weiß, dass Chinesen kleine Schwänze haben",1,train
...,...,...,...
10991,Mein bester Freund ist dunkelhäutig. Und?,0,train
10992,"Das ist eine gescheite Maßnahme, das sollten wir machen, aber das ist nicht das Ende der Fahnenstange.",0,dev
10993,Ein großer Teil der Belästigung von Schulschule ist auf die Islamisierung zurückzuführen,1,train
10994,"Araber, Juden, Schwarze, eine Lösung für alle #Endlösung",1,train


In [87]:
texts_list = []
texts_set = set()
for dataset in df_dict:
    for i in df_dict[dataset]["text"]:
        texts_list.append(i)
        if i in texts_set:
            print(dataset, i)
            continue
        texts_set.add(i)

len(texts_list), len(texts_set)

gahd24_de Muslimen die Fresse polieren, dass wäre es doch


(10996, 10995)

## Show descriptive stats

In [88]:
def descriptive_stats(df):
    n_total = df.shape[0]
    n_hate = df.label.sum()
    print("{} entries, of which {} ({:.2%}) are hateful.".format(n_total, n_hate, n_hate/n_total))
    return df.label.sum()/len(df), len(df)

for dataset in df_dict:
    print(dataset.upper())
    descriptive_stats(df_dict[dataset])
    display(df_dict[dataset].sample(5))
    print()

GAHD24_DE
10996 entries, of which 4666 (42.43%) are hateful.


,text,label,split
8976,"Darüber hinaus müssen sie für die Kosten (Schäden), die dieses System verursacht, nach wie vor nicht aufkommen.",0,train
6489,"Return of Monkey Island ist ein solches Spiel, zumindest kann es ein solches Spiel sein.",0,dev
3800,Der linke Geist hat Qualifikation mit Frauenquote ersetzt.,0,dev
1120,"Ältere Frauen oder Frauen in ländlichen Gebieten tragen zudem eine Burka, die den Körper vollständig verschleiert, wenn sie das Haus verlassen.",0,train
2733,"Wie geht es bei Pokémon Arceus weiter, wenn man die Strandrettiche gefunden hat?",0,train


In [89]:
new_dict = {}
# new_dict["xdomain_tr"] = df_dict["xdomain_tr"]
# new_dict["xdomain_en"] = df_dict["xdomain_en"]
new_dict["gahd24_de"] = df_dict["gahd24_de"]
# new_dict["xplain_en"] = df_dict["xplain_en"]
# new_dict["measure_en"] = df_dict["measure_en"]
# df_dict = new_dict

In [90]:
df_dict.keys()

dict_keys(['gahd24_de'])

In [91]:
df = new_dict['gahd24_de']

In [92]:
df.split.value_counts()

split
train    7701
dev      1649
test     1646
Name: count, dtype: int64

In [98]:
train_df = df[df['split'] == 'train']
dev_df = df[df['split'] == 'dev'] #df[df['split'] == 'validation']
test_df = df[df['split'] == 'test']
print(train_df.shape, dev_df.shape, test_df.shape)
# Adjust dev and test sizes
dev_to_test = dev_df.sample(n=2000 - len(test_df), random_state=123)  # Sample 354 from dev to add to test
remaining_dev = dev_df.drop(dev_to_test.index)
print(remaining_dev.shape, dev_to_test.shape)

# Create new test set by combining existing test and sampled dev
new_test_df = pd.concat([test_df, dev_to_test])
print(remaining_dev.shape, dev_to_test.shape, new_test_df.shape)

# Split the remaining dev into dev (500 instances) and extra_dev
# if len(remaining_dev) > 500:
new_dev_df = remaining_dev.sample(n=500, random_state=123)
extra_dev_df = remaining_dev.drop(new_dev_df.index)

train_df = train_df.drop(columns=['split'])
new_dev_df = new_dev_df.drop(columns=['split'])
new_test_df = new_test_df.drop(columns=['split'])
extra_dev_df = extra_dev_df.drop(columns=['split'])

(7701, 3) (1649, 3) (1646, 3)
(1295, 3) (354, 3)
(1295, 3) (354, 3) (2000, 3)


In [99]:
path = '/mounts/data/proj/faeze/data_efficient_hate/datasets/main/1_clean/gahd24_de/'
# df_dict['xplain_en'] = train_df
# Save the adjusted splits to separate files
train_df.to_csv(path+f'train_{train_df.shape[0]}.csv', index=False)
new_dev_df.to_csv(path + f'dev_{new_dev_df.shape[0]}.csv', index=False)
new_test_df.to_csv(path + f'test_{new_test_df.shape[0]}.csv', index=False)
extra_dev_df.to_csv(path + f'extra_dev_{extra_dev_df.shape[0]}.csv', index=False)


In [97]:
train_df.shape, new_dev_df.shape, extra_dev_df.shape, new_test_df.shape

((7701, 2), (500, 2), (0, 2), (2000, 2))

In [56]:
# set aside 2k from each dataset for testing and 500 for dev
# except for Ousidhoum in French and Arabic, where train set would otherwise be too small
# and for HASOC 20 and 21 in Hindi, where test splits are given

TEST_SIZE = 2000
DEV_SIZE = 500

for dataset in df_dict:
    if "ous19_fr" in dataset:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = 2000, random_state=123)
        devset, testset = train_test_split(devtest, test_size = 1500, random_state=123)
        devset.to_csv(PATH + f"/1_clean/{dataset}/dev_500.csv", index=False)
        testset.to_csv(PATH + f"/1_clean/{dataset}/test_1500.csv", index=False)
    elif "ous19_ar" in dataset:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = 1300, random_state=123)
        devset, testset = train_test_split(devtest, test_size = 1000, random_state=123)
        devset.to_csv(PATH + f"/1_clean/{dataset}/dev_300.csv", index=False)
        testset.to_csv(PATH + f"/1_clean/{dataset}/test_1000.csv", index=False)
    elif "has19_hi" in dataset or "has20_hi" in dataset: # use provided test sets
        df_dict[dataset][df_dict[dataset]["split"]=="test"].to_csv(PATH + f"/1_clean/{dataset}/test_{len(df_dict[dataset][df_dict[dataset]['split']=='test'])}.csv", index=False)
        df_dict[dataset], devset = train_test_split(df_dict[dataset][df_dict[dataset]["split"]=="train"], test_size = 500, random_state=123)
        devset.to_csv(PATH + f"/1_clean/{dataset}/dev_500.csv", index=False)
    elif 'xplain_en' not in dataset:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = TEST_SIZE+DEV_SIZE, random_state=123)
        devset, testset = train_test_split(devtest, test_size = TEST_SIZE, random_state=123)
        devset.to_csv(PATH + f"/1_clean/{dataset}/dev_{DEV_SIZE}.csv", index=False)
        testset.to_csv(PATH + f"/1_clean/{dataset}/test_{TEST_SIZE}.csv", index=False)
        
# export all data that is not test or dev, so we can use it for full sample training
for dataset in df_dict:
    df_dict[dataset].to_csv(PATH + f"/1_clean/{dataset}/train_{len(df_dict[dataset])}.csv", index=False)

In [57]:
# create differently-sized train portions from rest of data

SEEDS = 10 # for repeated experiments with different random data selection
N_RANGE = [10, 20, 30, 40, 50, 100, 200, 300, 400, 500, 1000, 2000, 3000, 4000, 5000, 10000, 20000]

for dataset in df_dict:
    print(dataset.upper())
    for n in N_RANGE:
        
        # save all splits for English test sets
        if n<len(df_dict[dataset]) and ("_en" in dataset):
            print(f"  saving n = {n} training set")
            for random_state in range(1, SEEDS+1):
                df_dict[dataset].sample(n, random_state = random_state).to_csv(PATH + f"/1_clean/{dataset}/train/train_{n}_rs{random_state}.csv",index=False)
        
        # save splits up to 2k for other datasets
        elif n<len(df_dict[dataset]) and n<=2000: 
            print(f"  saving n = {n} training set")
            for random_state in range(1, SEEDS+1):  
                df_dict[dataset].sample(n, random_state = random_state).to_csv(PATH + f"/1_clean/{dataset}/train/train_{n}_rs{random_state}.csv",index=False)
    
    print()

GAHD24_DE
  saving n = 10 training set
  saving n = 20 training set
  saving n = 30 training set
  saving n = 40 training set
  saving n = 50 training set
  saving n = 100 training set
  saving n = 200 training set
  saving n = 300 training set
  saving n = 400 training set
  saving n = 500 training set
  saving n = 1000 training set
  saving n = 2000 training set



In [20]:
df_dict.keys()

dict_keys(['gahd24_de'])